# Clase 2.3 - Testing Avanzado: Fixtures, Mocks y Parametrización

**Unidad:** 2 - Técnicas de Desarrollo  
**Conexión al Proyecto TaskFlow:** En esta clase aprenderemos técnicas avanzadas de testing para escribir tests más limpios, mantenibles y aislados del sistema TaskFlow.  

## 📋 Objetivos de Aprendizaje

Al finalizar esta clase, serás capaz de:
- [ ] Crear y usar fixtures de pytest
- [ ] Parametrizar tests para múltiples casos
- [ ] Usar mocks para aislar código bajo prueba
- [ ] Aplicar patch para reemplazar dependencias
- [ ] Escribir tests de servicios con repositorios mockeados

## 💻 Conexión con TaskFlow

En el sistema **TaskFlow**, los servicios dependen de repositorios. Hoy aprenderemos a testear servicios sin depender de la implementación real de los repositorios.

In [ ]:
# Importaciones necesarias para este notebook
import pytest
from unittest.mock import Mock, patch, MagicMock
from typing import List, Optional
from dataclasses import dataclass, field
from datetime import datetime

---

## 1. 🗼 Fixtures de pytest

**📐 Definición:** Una **fixture** es una función que prepara el entorno para ejecutar un test. Se usa para configurar datos, objetos o recursos que el test necesita.

**💡 ¿Por qué usar fixtures?**
- Reutilizar código de setup entre tests
- Manejar recursos (archivos, DB) automáticamente
- Hacer tests más legibles y mantenibles

### 📝 Sintaxis básica de una fixture

In [ ]:
# Ejemplo 1: Fixture simple

@pytest.fixture
def usuario():
    """Fixture que provee un usuario para tests."""
    return Usuario(id=1, username="jdoe", email="john@example.com")

# Definimos el modelo Usuario
@dataclass
class Usuario:
    """Modelo de usuario para TaskFlow."""
    id: Optional[int] = None
    username: str = ""
    email: str = ""
    
def test_usuario_fixture(usuario):
    """Test que usa la fixture usuario."""
    # La fixture se inyecta automáticamente
    assert usuario.username == "jdoe"
    assert usuario.email == "john@example.com"
    print("✅ Test pasó usando fixture")

### 📁 Scope de Fixtures

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║              SCOPES DE FIXTURES PYTEST                        ║
╠══════════════════════════════════════════════════════════════╣
║                                                               ║
║  function (default)  →  Se ejecuta una vez por test          ║
║  class             →  Se ejecuta una vez por clase de tests  ║
║  module            →  Se ejecuta una vez por módulo          ║
║  package           →  Se ejecuta una vez por paquete         ║
║  session           →  Se ejecuta una vez por sesión pytest   ║
║                                                               ║
║  Ejemplo:                                                     ║
║    @pytest.fixture(scope="module")  # Se comparte entre tests ║
║    def db_connection():            # del mismo módulo        ║
║        conn = connect_to_db()                                ║
║        yield conn                                             ║
║        conn.close()  # Cleanup después de todos los tests     ║
║                                                               ║
╚══════════════════════════════════════════════════════════════╝
""")

### 🔄 Fixtures con yield (Setup y Teardown)

In [ ]:
# Ejemplo 2: Fixture con yield para cleanup

@pytest.fixture
def repositorio_con_usuarios():
    """
    Fixture que crea un repositorio con usuarios de prueba.
    
    El código antes de 'yield' es el setup.
    El código después de 'yield' es el teardown/cleanup.
    """
    # SETUP: Crear repositorio y agregar datos
    repo = UsuarioRepository()
    repo.guardar(Usuario(username="user1", email="user1@example.com"))
    repo.guardar(Usuario(username="user2", email="user2@example.com"))
    
    # YIELD: Proveer el repositorio al test
    yield repo
    
    # TEARDOWN: Limpiar después del test
    repo._usuarios.clear()  # Limpiar la lista
    print("\n🧹 Cleanup: Repositorio limpiado después del test")

# Clase de repositorio simplificada para el ejemplo
class UsuarioRepository:
    def __init__(self):
        self._usuarios = []
        self._next_id = 1
    
    def guardar(self, usuario):
        if usuario.id is None:
            usuario.id = self._next_id
            self._next_id += 1
        self._usuarios.append(usuario)
    
    def listar_todos(self):
        return self._usuarios.copy()

def test_repositorio_fixture(repositorio_con_usuarios):
    """Test que usa la fixture con repositorio."""
    usuarios = repositorio_con_usuarios.listar_todos()
    
    assert len(usuarios) == 2
    assert usuarios[0].username == "user1"
    assert usuarios[1].username == "user2"
    print("\n✅ Test pasó: Repositorio tiene 2 usuarios")

### 🎯 Fixtures con parámetros

In [ ]:
# Ejemplo 3: Fixture parametrizable

@pytest.fixture
def usuario_con_params(request):
    """
    Fixture que crea un usuario con parámetros personalizados.
    
    Uso:
        @pytest.mark.parametrize("username", ["user1", "user2"])
        def test_algo(usuario_con_params):
            ...
    """
    # request.param viene del parametrize
    username = getattr(request, "param", "default_user")
    return Usuario(id=1, username=username, email=f"{username}@example.com")

# Test parametrizado que usa la fixture
@pytest.mark.parametrize("username", ["alice", "bob", "charlie"])
def test_usuarios_diferentes(usuario_con_params, username):
    """Test parametrizado con diferentes usernames."""
    assert usuario_con_params.username == username
    assert "@example.com" in usuario_con_params.email
    print(f"\n✅ Test pasó para usuario: {username}")

---

## 2. 🎲 Parametrización de Tests

**📐 Definición:** La **parametrización** permite ejecutar el mismo test con diferentes valores de entrada, sin repetir código.

### 📝 Sintaxis básica de parametrize

In [ ]:
# Ejemplo 4: Parametrización simple

@pytest.mark.parametrize("username, expected_valid", [
    ("jdoe", True),           # Username válido (más de 3 chars)
    ("ab", False),            # Username muy corto
    ("", False),              # Username vacío
    ("a", False),             # Un solo carácter
    ("very_long_username", True),  # Username largo válido
    ("123", True),            # Username con números válido
])
def test_validacion_username(username, expected_valid):
    """
    Test parametrizado de validación de username.
    
    Este test se ejecuta 6 veces, una por cada combinación
    de parámetros definida arriba.
    """
    es_valido = len(username) >= 3
    
    assert es_valido == expected_valid, (
        f"Username '{username}' debería ser {'válido' if expected_valid else 'inválido'}"
    )
    print(f"\n✅ Test pasó para username='{username}'")

### 🎯 Parametrización con múltiples valores

In [ ]:
# Ejemplo 5: Parametrización con múltiples parámetros

@pytest.mark.parametrize("email, expected_valid, error_msg", [
    ("user@example.com", True, ""),
    ("test@domain.co", True, ""),
    ("invalido", False, "no tiene @"),
    ("@nodomain.com", False, "no tiene usuario"),
    ("user@", False, "no tiene dominio"),
    ("", False, "está vacío"),
])
def test_validacion_email(email, expected_valid, error_msg):
    """
    Test parametrizado de validación de email.
    
    Cada test recibe: email, expected_valid, y error_msg
    """
    tiene_arroba = "@" in email
    tiene_usuario = email.split("@")[0] if "@" in email else False
    tiene_dominio = email.split("@")[1] if "@" in email and len(email.split("@")) > 1 else False
    
    es_valido = (
        len(email) > 0 and
        tiene_arroba and
        tiene_usuario and
        tiene_dominio
    )
    
    assert es_valido == expected_valid, (
        f"Email '{email}': {error_msg if error_msg else 'debería ser válido'}"
    )
    print(f"\n✅ Test pasó para email='{email}'")

### 🎨 IDs personalizados para tests parametrizados

In [ ]:
# Ejemplo 6: Parametrización con IDs personalizados

@pytest.mark.parametrize("username, email", [
    pytest.param("jdoe", "john@example.com", id="usuario_valido"),
    pytest.param("ab", "invalido", id="username_corto"),
    pytest.param("", "@example.com", id="username_vacio"),
    pytest.param("test", "no-arroba.com", id="email_sin_arroba"),
])
def test_creacion_usuario_con_ids(username, email):
    """
    Test parametrizado con IDs legibles en la salida de pytest.
    
    Al ejecutar pytest -v, verás:
        test_creacion_usuario_con_ids[usuario_valido] PASSED
        test_creacion_usuario_con_ids[username_corto] PASSED
        ...
    """
    try:
        usuario = Usuario(username=username, email=email)
        print(f"\n✅ Test [{username[:10]}...] pasó")
    except Exception as e:
        print(f"\n⚠️ Test [{username[:10]}...] falló esperadamente: {e}")

---

## 3. 🎭 Mocks con unittest.mock

**📐 Definición:** Un **mock** es un objeto simulado que imita el comportamiento de un objeto real. Se usa para aislar el código bajo prueba de sus dependencias.

**💡 ¿Por qué usar mocks?**
- Aislar el código bajo prueba
- Simular respuestas de dependencias externas (API, DB)
- Verificar interacciones sin ejecutar código real
- Hacer tests más rápidos y determinísticos

### 📝 Creación básica de Mocks

In [ ]:
# Ejemplo 7: Mock básico

def test_mock_basico():
    """Test que muestra cómo crear y usar un mock básico."""
    
    # Crear un mock
    mock_repo = Mock()
    
    # Configurar el mock para retornar un valor específico
    mock_repo.buscar_por_id.return_value = Usuario(id=1, username="jdoe")
    
    # Usar el mock como si fuera el objeto real
    usuario = mock_repo.buscar_por_id(1)
    
    # Verificar
    assert usuario.username == "jdoe"
    assert mock_repo.buscar_por_id.called  # Verificar que fue llamado
    assert mock_repo.buscar_por_id.call_count == 1  # Fue llamado 1 vez
    
    print("\n✅ Test pasó: Mock configurado y verificado")

### 🎯 Configurar comportamiento del Mock

In [ ]:
# Ejemplo 8: Configurar diferentes respuestas

def test_mock_con_respuestas_diferentes():
    """Test que muestra cómo configurar diferentes respuestas."""
    
    mock_repo = Mock(spec=UsuarioRepository)
    
    # Configurar respuestas diferentes según parámetros
    mock_repo.buscar_por_id.side_effect = [
        Usuario(id=1, username="user1"),  # Primera llamada
        Usuario(id=2, username="user2"),  # Segunda llamada
        None,  # Tercera llamada (usuario no encontrado)
    ]
    
    # Primera llamada
    u1 = mock_repo.buscar_por_id(1)
    assert u1.username == "user1"
    
    # Segunda llamada
    u2 = mock_repo.buscar_por_id(2)
    assert u2.username == "user2"
    
    # Tercera llamada
    u3 = mock_repo.buscar_por_id(999)
    assert u3 is None
    
    # Verificar número de llamadas
    assert mock_repo.buscar_por_id.call_count == 3
    
    print("\n✅ Test pasó: Mock con side_effect configurado")

### 🔧 Verificar llamadas al Mock

In [ ]:
# Ejemplo 9: Verificar interacciones con el mock

def test_verificar_interacciones_mock():
    """Test que muestra cómo verificar las llamadas al mock."""
    
    mock_repo = Mock()
    mock_repo.buscar_por_id.return_value = Usuario(id=1, username="jdoe")
    
    # Llamar al método
    mock_repo.buscar_por_id(1)
    mock_repo.buscar_por_id(1)
    mock_repo.buscar_por_username("jdoe")
    
    # Verificaciones
    assert mock_repo.buscar_por_id.called  # Fue llamado al menos una vez
    assert mock_repo.buscar_por_id.call_count == 2  # Fue llamado 2 veces
    
    # Verificar con qué parámetros fue llamado
    mock_repo.buscar_por_id.assert_called_with(1)  # Última llamada con 1
    mock_repo.buscar_por_id.assert_any_call(1)  # Al menos una llamada con 1
    
    # Verificar método específico fue llamado
    assert mock_repo.buscar_por_username.called
    mock_repo.buscar_por_username.assert_called_once_with("jdoe")
    
    # Verificar que un método NO fue llamado
    assert not mock_repo.guardar.called
    
    print("\n✅ Test pasó: Interacciones del mock verificadas")

---

## 4. 🩹 Patch: Reemplazar Dependencias

**📐 Definición:** **patch** es un decorador/context manager que reemplaza temporalmente un objeto con un mock durante la ejecución de un test.

### 📝 Usar patch como decorador

In [ ]:
# Ejemplo 10: Patch como decorador

# Clase bajo prueba
class UsuarioService:
    """
    Servicio de lógica de negocio de usuarios.
    Depende de UsuarioRepository.
    """
    def __init__(self, repositorio: UsuarioRepository):
        self.repositorio = repositorio
    
    def obtener_usuario(self, usuario_id: int) -> Optional[Usuario]:
        """
        Obtiene un usuario por ID.
        Lógica de negocio: usuario debe estar activo.
        """
        usuario = self.repositorio.buscar_por_id(usuario_id)
        
        if usuario and getattr(usuario, 'activo', True):
            return usuario
        
        return None

# Test con patch
@patch('__main__.UsuarioRepository')
def test_service_con_patch(mock_repo_class):
    """
    Test que usa patch para reemplazar UsuarioRepository.
    
    Nota: En producción, usar la ruta completa del módulo.
    """
    # Configurar la instancia del mock
    mock_instance = Mock()
    mock_instance.buscar_por_id.return_value = Usuario(
        id=1, username="jdoe", email="john@example.com"
    )
    mock_repo_class.return_value = mock_instance
    
    # Crear el servicio (usará el mock)
    service = UsuarioService(repositorio=mock_instance)
    
    # Ejecutar método bajo prueba
    usuario = service.obtener_usuario(1)
    
    # Verificar
    assert usuario is not None
    assert usuario.username == "jdoe"
    
    # Verificar que el repositorio fue llamado
    mock_instance.buscar_por_id.assert_called_once_with(1)
    
    print("\n✅ Test pasó: Patch de repositorio funcionó")

### 🎯 Patch con contexto

In [ ]:
# Ejemplo 11: Patch como contexto

def test_service_con_patch_contexto():
    """Test que usa patch como contexto con 'with'."""
    
    mock_repo = Mock()
    mock_repo.buscar_por_id.return_value = Usuario(
        id=1, username="jdoe", email="john@example.com"
    )
    
    # Crear servicio
    service = UsuarioService(repositorio=mock_repo)
    
    # El patch solo afecta dentro del bloque 'with'
    with patch('builtins.print') as mock_print:
        usuario = service.obtener_usuario(1)
        
        # Verificar que print fue llamado (si el servicio lo usa)
        # mock_print.assert_called()  # Si el servicio imprime
        
        assert usuario is not None
    
    print("\n✅ Test pasó: Patch con contexto funcionó")

---

## 5. 🎯 Ejercicio Práctico: Test de Servicio con Mock

Completa los tests para `UsuarioService` usando mocks del repositorio.

In [ ]:
# Implementación del servicio
class UsuarioService:
    """
    Servicio de lógica de negocio de usuarios.
    """
    
    def __init__(self, repositorio: UsuarioRepository):
        self.repositorio = repositorio
    
    def crear_usuario(self, username: str, email: str) -> Usuario:
        """
        Crea un nuevo usuario.
        
        Lógica de negocio:
        - Username debe ser único
        - Email debe ser único
        - Username mínimo 3 caracteres
        
        Returns:
            El usuario creado
            
        Raises:
            ValueError: Si las validaciones fallan
        """
        # Validaciones
        if len(username) < 3:
            raise ValueError("Username debe tener al menos 3 caracteres")
        
        # Verificar unicidad de username
        existe = self.repositorio.buscar_por_username(username)
        if existe:
            raise ValueError(f"Username '{username}' ya existe")
        
        # Crear usuario
        usuario = Usuario(username=username, email=email)
        self.repositorio.guardar(usuario)
        
        return usuario
    
    def obtener_usuario(self, usuario_id: int) -> Optional[Usuario]:
        """Obtiene un usuario por ID."""
        return self.repositorio.buscar_por_id(usuario_id)

### 🔴 Escribe los tests

In [ ]:
# Tests para UsuarioService con mocks

def test_crear_usuario_exitoso():
    """
    Test: Crear usuario exitosamente.
    
    TODO: Completar usando mock del repositorio.
    """
    # Arrange
    mock_repo = Mock()
    mock_repo.buscar_por_username.return_value = None  # Username disponible
    
    service = UsuarioService(repositorio=mock_repo)
    
    # Act
    usuario = service.crear_usuario("jdoe", "john@example.com")
    
    # Assert
    assert usuario.username == "jdoe"
    assert mock_repo.guardar.called
    
    print("\n✅ Test 1 pasó: Usuario creado exitosamente")

def test_crear_usuario_username_duplicado_falla():
    """
    Test: Crear usuario con username duplicado falla.
    
    TODO: Completar usando mock del repositorio.
    """
    # Arrange
    mock_repo = Mock()
    mock_repo.buscar_por_username.return_value = Usuario(
        id=1, username="jdoe", email="old@example.com"
    )
    
    service = UsuarioService(repositorio=mock_repo)
    
    # Act & Assert
    try:
        service.crear_usuario("jdoe", "new@example.com")
        assert False, "Debería haber lanzado ValueError"
    except ValueError as e:
        assert "ya existe" in str(e).lower()
    
    print("\n✅ Test 2 pasó: Username duplicado rechazado")

def test_crear_usuario_username_corto_falla():
    """
    Test: Crear usuario con username corto falla.
    
    TODO: Completar este test.
    """
    # TODO: Completar
    pass

### ✅ Ejecutar tests

In [ ]:
# Ejecutar todos los tests

def suite_completa_servicio():
    """Ejecuta todos los tests del servicio."""
    
    try:
        test_crear_usuario_exitoso()
        test_crear_usuario_username_duplicado_falla()
        # test_crear_usuario_username_corto_falla()  # TODO
        print("\n🚀 ¡Suite de tests del servicio completada!")
    except Exception as e:
        print(f"\n❌ Error: {e}")

# Ejecutar
suite_completa_servicio()

---

## 6. 📊 Resumen Visual: Testing Avanzado

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║            TESTING AVANZADO - RESUMEN                         ║
╠══════════════════════════════════════════════════════════════╣
║                                                               ║
║  🗼 FIXTURES                                                   ║
║     @pytest.fixture                                          ║
║     def usuario():                                            ║
║         return Usuario(...)                                   ║
║                                                               ║
║  🎲 PARAMETRIZACIÓN                                            ║
║     @pytest.mark.parametrize("x,y", [(1,2), (3,4)])         ║
║     def test_algo(x, y):                                      ║
║         assert x < y                                          ║
║                                                               ║
║  🎭 MOCKS                                                      ║
║     mock = Mock()                                             ║
║     mock.metodo.return_value = resultado                     ║
║     assert mock.metodo.called                                 ║
║                                                               ║
║  🩹 PATCH                                                      ║
║     @patch('modulo.Clase')                                    ║
║     def test_algo(mock_clase):                                ║
║         ...                                                   ║
║                                                               ║
╚══════════════════════════════════════════════════════════════╝
""")

---

## 📝 Resumen de la Clase

### 📋 Conceptos Clave

| Concepto | Descripción | Ejemplo |
|----------|-------------|--------- |
| **Fixture** | Función de setup/teardown | `@pytest.fixture` |
| **Scope** | Alcance de la fixture | `function`, `class`, `module` |
| **Yield** | Punto de cleanup en fixture | `yield repo` luego cleanup |
| **Parametrize** | Múltiples casos en un test | `@pytest.mark.parametrize` |
| **Mock** | Objeto simulado | `Mock()` |
| **Patch** | Reemplazar temporalmente | `@patch('modulo.Clase')` |
| **side_effect** | Respuestas secuenciales | `mock.foo.side_effect = [1, 2]` |

### 🤝 Conexión con TaskFlow

Hemos aprendido a:
- Crear fixtures para reutilizar setup
- Parametrizar tests para múltiples casos
- Usar mocks para aislar servicios
- Aplicar patch para reemplazar dependencias

En la próxima clase aprenderemos **BDD con behave**: Escenarios Gherkin y pruebas de comportamiento.

---

## 🚀 Reto Final (Opcional)

Crea una suite de tests completa para `ProyectoService`:
1. Fixture para repositorio mockeado
2. Tests parametrizados para validaciones
3. Tests con patch para dependencias externas
4. Verificación de interacciones con mocks

In [ ]:
# Tu reto aquí

@pytest.fixture
def proyecto_service():
    # TODO: Crear fixture para ProyectoService
    pass

@pytest.mark.parametrize("nombre, expected_valid", [
    # TODO: Agregar casos de prueba
])
def test_validacion_nombre_proyecto(nombre, expected_valid):
    # TODO: Completar test
    pass

---

**📈 Próxima clase:** 02-04 Introducción a BDD

Aprenderemos a:
- Escribir escenarios en formato Gherkin
- Implementar pasos de BDD con behave
- Conectar escenarios con código Python
- Ejecutar pruebas BDD desde terminal